In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
imgdim = 512#192#224
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']
classes = CLASSES


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim

from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [4]:
# !pip install timm
#%pip install albumentations
#%pip install seaborn

In [5]:
import pandas as pd
from scipy import stats
import numpy as np
import glob
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
# import timm
import random
import time
import copy
from operator import itemgetter

from collections import OrderedDict, namedtuple
import joblib

import logging
import sys

import PIL
import cv2
import albumentations as A
import io
import IPython.display as display

# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.debug.metrics as met
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.utils.utils as xu

import torchvision
# from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics, model_selection

import warnings
warnings.filterwarnings("ignore");

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
datadir = r"/kaggle/input/tpu-getting-started/tfrecords-jpeg-{0}x{0}//".format(imgdim)
train_files = glob.glob(datadir + '/train/*.tfrec')
val_files = glob.glob(datadir + '/val/*.tfrec')
test_files = glob.glob(datadir + '/test/*.tfrec')
# augment train set with val set, rebalancing it
train_files += val_files[4:]
val_files = val_files[:4]
print(len(train_files), len(val_files), len(test_files))

28 4 16


In [7]:
low_freq_targets = [1, 32, 19, 27, 61, 101, 92, 66, 85, 84, 100, 11, 31, 26, 15, 34, 2, 97, 99, 3, 98, 23, 60, 33, 6, 54, 63, 35, 20, 65, 58, 44, 37, 38]
med_freq_targets = [57, 70, 22, 64, 8, 56, 25, 18, 21, 30, 16, 78, 40, 12, 81, 51, 9, 89, 90, 96, 39, 88, 59, 41, 69, 29, 7, 24, 5, 17, 42, 55, 36, 62]
aug_targets = low_freq_targets + med_freq_targets

In [8]:
def parse_tfrec_data(files, test=False):
    if not test: 
        feature_description = {
            'class': tf.io.FixedLenFeature([], tf.int64),
            'id': tf.io.FixedLenFeature([], tf.string),
            'image': tf.io.FixedLenFeature([], tf.string),
        }
    else:
        feature_description = {
        'id': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    parse_image_f = lambda x: tf.io.parse_single_example(x, feature_description)

    ids = []
    images = []
    if not test:
        cl = []

    for i in tqdm(files):
        image_dataset = tf.data.TFRecordDataset(i)
        image_dataset = image_dataset.map(parse_image_f)

        ids_ = [str(id_features['id'].numpy())[2:-1] for id_features in image_dataset] # [2:-1] is done to remove b' from 1st and 'from last in train id names
        ids = ids + ids_
        
               
        images_ = [image_features['image'].numpy() for image_features in image_dataset]
        images = images + images_
                

        if not test:
            cl_ = [int(class_features['class'].numpy()) for class_features in image_dataset]
            cl = cl + cl_
            #print(i, len(ids))
            [ids.append(j) for j,k in zip(ids_, cl_) if k in aug_targets]# augment low freq targets
            #print(i, len(ids))
            [images.append(j) for j,k in zip(images_, cl_) if k in aug_targets]# augment low freq targets
            [cl.append(j) for j in cl_ if j in aug_targets]# augment low freq targets
            
            
    if test:
        return ids, images
    else:
        return ids, cl, images

In [9]:
#train_ids, train_cl, train_images = parse_tfrec_data(train_files)
#val_ids, val_cl, val_images = parse_tfrec_data(val_files)
test_ids, test_images = parse_tfrec_data(test_files, test=True)

100%|██████████| 16/16 [00:20<00:00,  1.30s/it]


In [10]:
#print(f'''# train images: {len(train_ids)}
# val images: {len(val_ids)}
# test images {len(test_ids)}''' )

In [11]:
#print(train_ids[0])
#print(train_cl[0])
#sample_img = PIL.Image.open(io.BytesIO(train_images[0]))
#sample_img

In [12]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [13]:
class FlowerDataset(Dataset):
    
    def __init__(self, ids, images, cl, transforms, test=False):
        
        self.ids = ids
        self.images = images
        if not test:
            self.cl = cl
        self.transforms = transforms
        self.is_test = test
    
    def __len__(self):
        return len(self.ids)
        
    def __getitem__(self, idx):
        
        img = self.images[idx]
        img = PIL.Image.open(io.BytesIO(img))
        img = np.array(img)
        img = self.transforms(image=img)['image']
        
        if self.is_test:
            return img, -1, self.ids[idx]
        return img, int(self.cl[idx]), self.ids[idx]
     

In [14]:
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([A.HorizontalFlip(p=0.4),
                           A.VerticalFlip(p=0.1),
                           A.ShiftScaleRotate(p=0.1),
                            A.ColorJitter(p=0.1),
                           A.OneOf([A.RandomShadow(p=1.), A.RandomSunFlare(src_radius=20, p=1.),], p=0.1),
                            A.RandomFog(p=0.1),
                              A.Sharpen(alpha=(0.1, 0.3), lightness=(0.5, 1.0), always_apply=False, p=0.1),
                              A.Blur(blur_limit=(2,5), p=0.1),
                           A.GaussNoise(var_limit=120. , p=0.2),
                           A.RandomBrightness(limit=0.1, p=0.1),
                            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                           ToTensorV2()
                              
    ])

val_transforms = A.Compose([A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2() ])
test_transforms = A.Compose([A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()])

In [15]:
#train_dataset = FlowerDataset(train_ids, train_images, train_cl, transforms=train_transforms)
#val_dataset = FlowerDataset(val_ids, val_images, val_cl, transforms=val_transforms)
test_dataset = FlowerDataset(test_ids, test_images, None, transforms=test_transforms, test=True)

#train_loader = DataLoader(train_dataset, batch_size=16, num_workers=4, shuffle=True, pin_memory=True)
#val_loader = DataLoader(val_dataset, batch_size=16, num_workers=4,  shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, num_workers=4,  shuffle=True, pin_memory=True)

In [16]:
"""
train_iter = iter(train_loader)
img = next(train_iter)
for img2show in img[0][:10].permute(0, 2, 3, 1):
    visualize(img2show.numpy())
"""

'\ntrain_iter = iter(train_loader)\nimg = next(train_iter)\nfor img2show in img[0][:10].permute(0, 2, 3, 1):\n    visualize(img2show.numpy())\n'

In [17]:
from torch.autograd import Variable

# Function to save the model
def saveModel():
    path = "./myFirstModel.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    model.eval()
    accuracy = 0.0
    total = 0.0
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for fn in val_files:
            
            val_ids, val_cl, val_images = parse_tfrec_data([fn])
            val_dataset = FlowerDataset(val_ids, val_images, val_cl, transforms=val_transforms)
            val_loader = DataLoader(val_dataset, batch_size=16, num_workers=4, shuffle=True, pin_memory=True)

            for batch, labels, _ in val_loader:

                batch = batch.cuda()
                labels = labels.cuda()
                out = model(batch)
                pred_labels = out.argmax(dim=1)
                accuracy += float((pred_labels == labels).sum())
                total += labels.shape[0]

    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return accuracy


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs, lr_, model):
    
    best_accuracy = 0.0

    # Define your execution device
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model = model.cuda()
    model = model.train()
    print('model on device')
    #lr_ = 0.001
    train_loss, train_acc = [], []
    test_loss, test_acc = [], []

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.
        num_correct = 0.
        num_total = 0.
        
        for fn in train_files:
            
            train_ids, train_cl, train_images = parse_tfrec_data([fn])
            train_dataset = FlowerDataset(train_ids, train_images, train_cl, transforms=train_transforms)
            train_loader = DataLoader(train_dataset, batch_size=16, num_workers=4, shuffle=True, pin_memory=True)

            for batch, labels, _ in train_loader:

                batch = batch.cuda()
                labels = labels.cuda()

                optimizer.zero_grad()

                out = model(batch)
                pred_labels = out.argmax(dim=1)
                num_correct += float((pred_labels == labels).sum())
                num_total += labels.shape[0]

                loss = criterion(out, labels)
                running_loss += loss
                loss.backward()
                optimizer.step()


        epoch_loss = running_loss / num_total
        epoch_acc = num_correct / num_total
        
        train_loss.append(epoch_loss.data.cpu())
        train_acc.append(epoch_acc)
        
        #t_loss, t_acc = eval_model(model, criterion, test_loader, cuda=cuda)
        
        #test_loss.append(t_loss.data.cpu())
        #test_acc.append(t_acc)
                
        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        test_acc.append(accuracy)
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        #val_result = valClasses(classes=classes)
        #print('For epoch', epoch+1,'the val accuracy over the whole test set is %d %%' % (val_result))
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy
            
        #for param in model.parameters():
        #    param.requires_grad=True
            
        #scheduler.step()
        if (epoch > 2): # if beyond 2 epochs and 
            if test_acc[-1] > test_acc[-2]: lr_ *= 0.5
        else: lr_ = 0.001

In [18]:
import torch
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
import tensorflow as tf
import matplotlib.pyplot as plt
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
#tf.compat.v1.per_process_gpu_memory_fraction = 0.001

In [19]:
# Function to test what classes performed well
def testClasses(classes):
    number_of_labels = len(classes)
    class_correct = list(0. for i in range(number_of_labels))
    class_total = list(0. for i in range(number_of_labels))
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    with torch.no_grad():

        for batch, labels, _ in val_loader:

            batch = batch.cuda()
            labels = labels.cuda()
            out = model(batch)
            pred_labels = out.argmax(dim=1)
            c = (pred_labels == labels)
            
            for i in range(batch_size):
                try:
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1
                except: pass
    for i in range(number_of_labels):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [20]:
#comparison = pd.read_csv('submission (4).csv')

In [21]:
import torch.nn as nn
import torchvision
import torch.nn.functional as F
# Instantiate a neural network model 
model = torchvision.models.efficientnet_b4(pretrained=True)
for (n, p), param in zip(model.named_parameters(), model.parameters()):
    if any([n.startswith(i) for i in ['features.7', 'features.8', 'classifier']]):
        param.requires_grad=True
        print(n)
    else: param.requires_grad=False
    
model.fc = nn.Linear(2048, 104)


Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 191MB/s]


features.7.0.block.0.0.weight
features.7.0.block.0.1.weight
features.7.0.block.0.1.bias
features.7.0.block.1.0.weight
features.7.0.block.1.1.weight
features.7.0.block.1.1.bias
features.7.0.block.2.fc1.weight
features.7.0.block.2.fc1.bias
features.7.0.block.2.fc2.weight
features.7.0.block.2.fc2.bias
features.7.0.block.3.0.weight
features.7.0.block.3.1.weight
features.7.0.block.3.1.bias
features.7.1.block.0.0.weight
features.7.1.block.0.1.weight
features.7.1.block.0.1.bias
features.7.1.block.1.0.weight
features.7.1.block.1.1.weight
features.7.1.block.1.1.bias
features.7.1.block.2.fc1.weight
features.7.1.block.2.fc1.bias
features.7.1.block.2.fc2.weight
features.7.1.block.2.fc2.bias
features.7.1.block.3.0.weight
features.7.1.block.3.1.weight
features.7.1.block.3.1.bias
features.8.0.weight
features.8.1.weight
features.8.1.bias
classifier.1.weight
classifier.1.bias


In [22]:
#torch.set_grad_enabled(True)
from torch.optim import lr_scheduler, Adam

optimizer = Adam(model.parameters(), lr=0.001)#, weight_decay=0.00002) 
loss_fn = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

# Let's build our model
train(6, 0.005, model)
model.fc = nn.Linear(2048, 104)
train(6, 0.001, model)
model.fc = nn.Linear(2048, 104)
train(8, 0.0002, model)

print('Finished Training')

model on device


100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


For epoch 1 the test accuracy over the whole test set is 87 %


100%|██████████| 1/1 [00:00<00:00,  1.98it/s]


For epoch 2 the test accuracy over the whole test set is 86 %


...

For epoch 6 the test accuracy over the whole test set is 91 %


100%|██████████| 1/1 [00:00<00:00,  2.22it/s]


For epoch 7 the test accuracy over the whole test set is 91 %


100%|██████████| 1/1 [00:00<00:00,  2.12it/s]


For epoch 8 the test accuracy over the whole test set is 91 %
Finished Training


In [23]:
# Test which classes performed well
testAccuracy()
# Function to test what classes performed well
#testClasses(classes=classes)

100%|██████████| 1/1 [00:00<00:00,  1.89it/s]


91.97019867549669

In [24]:
def make_submission(model, test_loader, cuda=True):

    results=[]
    for batch, _, ids in tqdm(test_loader):
        with torch.no_grad():
            batch = batch.cuda()
            model.eval()
            out = model(batch)
            pred_labels = torch.argmax(out.data.cpu(), dim=1)
            rows = list(
                zip(
                    list(ids), list(pred_labels.numpy().tolist())
                )
            )
            results.append(pd.DataFrame(rows, columns=['id', 'label']))
    result_df = pd.concat(results)
    result_df['label'] = result_df['label'].astype(int)
    return result_df


In [25]:
submission = make_submission(model, test_loader)

100%|██████████| 462/462 [01:27<00:00,  5.29it/s]


In [26]:
submission.to_csv('submission.csv', index=False)

In [27]:
# Let's load the model we just created and test the accuracy per label
from datetime import datetime
torch.save(model,'CV_petaltometal_model_{}'.format(datetime.now().strftime('%Y%m%d%H%M')))
print('model saved')

model saved
